This note shows how I collect data from various papers.

In [1]:
import sys

import numpy as np

from astropy.table import Table, join, vstack, Column, unique
from astropy.coordinates import SkyCoord
import astropy.units as u

Lists of pulsars in the *Gaia* EDR3 and PRS$\pi$

In [2]:
edr3_psr = Table.read("../data/gedr3_psr_name.dat",
                      format="ascii",
                      names=[
                          "1", "psr_bname", "2", "psr_name", "3", "4", "5",
                          "6", "7", "8", "9"
                      ], include_names=["psr_name", "psr_bname"])
vlbi_psr = Table.read("../data/psrpi_psr_name.dat", format="ascii",
                      names=[
                          "1", "psr_bname", "2", "psr_name", "3", "4", "5",
                          "6", "7", "8", "9"
                      ], include_names=["psr_name", "psr_bname"])

Create a dict to translate B1950 name into J2000 name.

In [3]:
psr_name_map ={
    "B1855+09": "J1857+0943",
    "B1937+21": "J1939+2134",
    "B1953+29": "J1955+2908",
    "B1957+20": "J1959+2048",
    "B1534+12": "J1537+1155",
}

Define some functions to used later.

In [4]:
def guess_sig(dat_str):
    """Guess significant value for parameter
    """

    if "." in dat_str:
        sig_dig = len(dat_str.strip().split(".")[1])
    else:
        sig_dig = 0

    return sig_dig


def parse_radec_str(ra_line, dec_line):
    """Convert RA and dec measurement from string into values
    """

    # RA and its error
    ra_str, ra_err_str = ra_line.split("(")
    ra_sig_dig = guess_sig(ra_str)
    # second -> mas
    ra_err = float(ra_err_str.split(")")[0]) * 10**(3 - ra_sig_dig) * 15

    # Decl. and its error
    dec_str, dec_err_str = dec_line.split("(")
    dec_sig_dig = guess_sig(dec_str)
    # arcsec -> mas
    dec_err = float(dec_err_str.split(")")[0]) * 10**(3 - dec_sig_dig)

    ra_dec = SkyCoord(ra_str, dec_str, unit=(u.hourangle, u.degree))
    ra = ra_dec.ra.deg
    dec = ra_dec.dec.deg

    # Correct for the cos(decl.)
    ra_err = ra_err * np.cos(ra_dec.dec.rad)

    return ra, dec, ra_err, dec_err


def parse_radec_str2(ra_str, dec_str, ra_err, dec_err):
    """Convert RA and dec measurement from string into values
    """

    # RA and decl.
    ra_dec = SkyCoord(ra_str, dec_str, unit=(u.hourangle, u.degree))
    ra = ra_dec.ra.deg
    dec = ra_dec.dec.deg

    # RA and its error
    ra_sig_dig = guess_sig(ra_str)
    dec_sig_dig = guess_sig(dec_str)

    # second -> mas
    ra_err = ra_err * 10**(3 - ra_sig_dig) * 15 * np.cos(ra_dec.dec.rad)

    # arcsec -> mas
    dec_err = dec_err * 10**(3 - dec_sig_dig)

    return ra, dec, ra_err, dec_err


def parse_radec_str3(ra_str, dec_str, ra_err, dec_err):
    """Convert RA and dec measurement from string into values
    """

    # RA and decl.
    ra_dec = SkyCoord(ra_str, dec_str, unit=(u.hourangle, u.degree))
    ra = ra_dec.ra.deg
    dec = ra_dec.dec.deg

    # second -> mas
    ra_err = ra_err * 10**3 * 15 * np.cos(ra_dec.dec.rad)

    # arcsec -> mas
    dec_err = dec_err * 10**3

    return ra, dec, ra_err, dec_err


def parse_pmp_str(dat_line):
    """Convert measurement for PM and parallax from string into value
    """

    if len(dat_line):
        val_str, val_err_str = dat_line.split("(")

        val = float(val_str)
        val_err = float(val_err_str.split(")")[0])
    else:
        val, val_err = 0, 0

    return val, val_err


def parse_pmp_str2(dat_line):
    """Convert measurement for PM and parallax from string into value
    """

    if len(dat_line):
        val_str, val_err_str = dat_line.split("(")

        val = float(val_str)
        val_sig_dig = guess_sig(val_str)
        val_err = float(val_err_str.split(")")[0]) * 10**( - val_sig_dig)
    else:
        val, val_err = 0, 0

    return val, val_err

# 1 Collect timing solution from literatures

## Paper 1: [Timing parameters for 59 pulsars.](https://ui.adsabs.harvard.edu/abs/1993MNRAS.262..449S/abstract) 

by Siegman, B. C. ; Manchester, R. N. ; Durdin, J. M., where Table 1 and 2 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

I assume the position epoch to be the mean epoch of observations.

In [5]:
smd93_tab = Table.read(
    "../data/smd93-table1.dat",
    format="ascii.fixed_width_no_header",
    col_starts=[
        10,
        21,
        34,
        41,
        55,
    ],
    col_ends=[
        19,
        33,
        39,
        53,
        59,
    ],
    names=["psr_name", "ra_str", "ra_err", "dec_str", "dec_err"])

smd93_tab2 = Table.read(
    "../data/smd93-table2.dat",
    format="ascii.no_header")

psr_num = len(smd93_tab)
psr_name = []
pos_epoch = np.zeros(psr_num)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["smd93"] * psr_num
comment = [""] * psr_num

for i in range(psr_num):
    
    psr_name.append("J%s" % smd93_tab["psr_name"][i])

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str3(
        smd93_tab["ra_str"][i], smd93_tab["dec_str"][i],
        smd93_tab["ra_err"][i], smd93_tab["dec_err"][i])
    
    # Position epoch
    epoch1, epoch2 = smd93_tab2["col9"][i].split("-")
    pos_epoch[i] = (float(epoch1) + float(epoch2)) / 2

smd93_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

## Paper 2: [Timing Behavior of 96 Radio Pulsars](https://ui.adsabs.harvard.edu/abs/1994ApJ...422..671A/abstract) 

by Arzoumanian, Z. ; Nice, D. J. ; Taylor, J. H. ; Thorsett, S. E., where Table 1 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

In [6]:
ant94_tab = Table.read("../data/ant94_table1.dat",
                        format="ascii.no_header",
                        data_start=5,
                        names=[
                            "psr_bname", "ra_str", "ra_err", "dec_str",
                            "dec_err", "dm", "dm_err", "p", "p_err", "p1",
                            "p1_err", "pos_epoch", "w", "r", "8"
                        ],
                        include_names=[
                            "psr_bname", "ra_str", "ra_err", "dec_str",
                            "dec_err", "pos_epoch"
                        ])

# Convert the data format.

psr_num = len(ant94_tab)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["ant94"] * psr_num
comment = [""] * psr_num

for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str2(
        ant94_tab["ra_str"][i], ant94_tab["dec_str"][i],
        ant94_tab["ra_err"][i], ant94_tab["dec_err"][i])

ant94_tab = Table([
    ant94_tab["psr_bname"], ant94_tab["pos_epoch"], eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
                  names=[
                      "psr_bname", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

# # Remove PSRs without RA or dec measurements
mask = ((ant94_tab["ra_err"] != 0) & (ant94_tab["dec_err"] != 0))
ant94_tab = ant94_tab[mask]

## Paper 3: [Timing measurements for 45 pulsars.](https://ui.adsabs.harvard.edu/abs/1995A&A...293..795J/abstract) 

by Johnston, S. ; Manchester, R. N. ; Lyne, A. G. ; Kaspi, V. M. ; D'Amico, N., where Table 1 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

In [7]:
jml95_tab = Table.read(
    "../data/jml+95_table.dat",
    format="ascii.fixed_width_no_header",
    data_start=2,
    col_starts=[2, 24, 40, 97],
    col_ends=[11, 39, 54, 104],
    names=["psr_jname", "ra_str", "dec_str", "pos_epoch"])

psr_num = len(jml95_tab)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["jml+95"] * psr_num
comment = [""] * psr_num

psr_name = ["J" + psr_namei.strip() for psr_namei in jml95_tab["psr_jname"]]

for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        jml95_tab["ra_str"][i], jml95_tab["dec_str"][i])

jml95_tab = Table([
    psr_name, jml95_tab["pos_epoch"], eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 4: [Discovery of Three Binary Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/1994ApJ...425L..41B/abstract) 

by Bailes, M. ; Harrison, P. A. ; Lorimer, D. R. ; Johnston, S. ; Lyne, A. G. ; Manchester, R. N. ; D'Amico, N. ; Nicastro, L. ; Tauris, T. M. ; Robinson, C., where Table 1 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

In [8]:
bhl94_tab = Table.read("../data/bhl+94_table.dat", format="ascii.csv")

psr_num = len(bhl94_tab)

ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["bhl+94"] * psr_num
comment = [""] * psr_num


for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        bhl94_tab["ra_str"][i], bhl94_tab["dec_str"][i])

bhl94_tab = Table([
    bhl94_tab["psr_name"], bhl94_tab["pos_epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 5: [The New Northern Cross Pulsar System: Four Years of Pulsar Timing Observations](https://ui.adsabs.harvard.edu/abs/1996ApJS..106..611D/abstract) 

by D'Amico, N. ; Grueff, G. ; Montebugnoli, S. ; Maccaferri, A. ; Cattani, A. ; Bortolotti, C. ; Nicastro, L. ; Fauci, F. ; Tomassetti, G. ; Roma, M. ; Ambrosini, R. ; Rodriguez, E., where Table 1 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

In [9]:
dgm96_tab = Table.read(
    "../data/dgm+96_table1.dat",
    format="ascii.fixed_width_no_header",
    data_start=2,
    col_starts=[1, 25, 39, 43, 58, 94],
    col_ends=[9, 37, 41, 55, 59, 99],
    names=["psr_jname", "ra_str", "ra_err", "dec_str", "dec_err", "pos_epoch"])

psr_num = len(dgm96_tab)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["dgm+96"] * psr_num
comment = [""] * psr_num

psr_name = ["J" + psr_namei.strip() for psr_namei in dgm96_tab["psr_jname"]]
pos_epoch = [ float(pos_epochi) for pos_epochi in dgm96_tab["pos_epoch"]]

for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str2(
        dgm96_tab["ra_str"][i], dgm96_tab["dec_str"][i],
        dgm96_tab["ra_err"][i], dgm96_tab["dec_err"][i])

dgm96_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 6: [The Parkes Southern Pulsar Survey - III. Timing of long-period pulsars](https://ui.adsabs.harvard.edu/abs/1998MNRAS.297...28D/abstract) 
by D'Amico, N. ; Stappers, B. W. ; Bailes, M. ; Martin, C. E. ; Bell, J. F. ; Lyne, A. G. ; Manchester, R. N., where Table 1 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

In [10]:
dsb98_tab = Table.read(
    "../data/dsb+98_table1.dat",
    format="ascii.fixed_width_no_header",
    data_start=2,
    col_starts=[3, 15, 33, 85],
    col_ends=[12, 32, 50, 93],
    names=["psr_jname", "ra_str", "dec_str", "pos_epoch"])

psr_num = len(dsb98_tab)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["dsb+98"] * psr_num
comment = [""] * psr_num

psr_name = ["J" + psr_namei.strip() for psr_namei in dsb98_tab["psr_jname"]]

for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        dsb98_tab["ra_str"][i], dsb98_tab["dec_str"][i])

dsb98_tab = Table([
    psr_name, dsb98_tab["pos_epoch"], eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 7: [New timing parameters and positions for 16 southern radio pulsars.](https://ui.adsabs.harvard.edu/abs/1998MNRAS.297...28D/abstract) 
by Indrani, C. ; Deshpande, A. A. ; Balasubramanian, V., where Table 1 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

In [11]:
idb98_tab = Table.read(
    "../data/idb98_table1.dat",
    format="ascii.fixed_width_no_header",
    data_start=3,
    col_starts=[3, 16, 33, 83],
    col_ends=[13, 31, 48, 94],
    names=["psr_jname", "ra_str", "dec_str", "pos_epoch"])

psr_num = len(idb98_tab)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["idb98"] * psr_num
comment = [""] * psr_num

for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        idb98_tab["ra_str"][i], idb98_tab["dec_str"][i])

idb98_tab = Table([
    idb98_tab["psr_jname"], idb98_tab["pos_epoch"], eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 8: [Millisecond pulsar velocities](https://ui.adsabs.harvard.edu/abs/1999MNRAS.307..925T/abstract) 

by Toscano, M. ; Sandhu, J. S. ; Bailes, M. ; Manchester, R. N. ; Britton, M. C. ; Kulkarni, S. R. ; Anderson, S. B. ; Stappers, B. W., where Table 1 can be extracted directly from the paper.

The timing analysis uses **DE200**.

In [12]:
tsb99_tab = Table.read("../data/tsb+99_table.dat", format="ascii.csv")

psr_num = len(tsb99_tab)

ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["tsb+99"] * psr_num
comment = [""] * psr_num


for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        tsb99_tab["ra_str"][i], tsb99_tab["dec_str"][i])

    pmra[i], pmra_err[i] = parse_pmp_str2(tsb99_tab["pmra_str"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str2(tsb99_tab["pmdec_str"][i])

tsb99_tab = Table([
    tsb99_tab["psr_name"], tsb99_tab["pos_epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 9: [Pulsar timing at Urumqi Astronomical Observatory: observing system and results](https://ui.adsabs.harvard.edu/abs/2001MNRAS.328..855W/abstract) 
by Wang, N. ; Manchester, R. N. ; Zhang, J. ; Wu, X. J. ; Yusup, A. ; Lyne, A. G. ; Cheng, K. S. ; Chen, M. Z., where Table 1 can be downloaded from [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/328/855).

The timing analysis uses **DE200**.

In [13]:
wmz01_tab = Table.read("../data/J_MNRAS_328_855_table1.dat.fits.gz")

psr_num = len(wmz01_tab)

# Bytes to str
psr_name = ["J" + psr_namei.strip() for psr_namei in wmz01_tab["PSRJ"]]

# Position epoch
pos_epoch = np.ones(psr_num) * 51700

dec_sign = np.where(wmz01_tab["DE1-"] == "-", -1, 1)

ra = (wmz01_tab["RA1h"] + wmz01_tab["RA1m"] / 60 + wmz01_tab["RA1s"] / 3600) * 15
dec = dec_sign * (wmz01_tab["DE1d"] + wmz01_tab["DE1m"] / 60 +
                  wmz01_tab["DE1s"] / 3600)

# mas
ra_err = wmz01_tab["e_RAs"] * 15 * np.cos(np.deg2rad(dec))
dec_err = wmz01_tab["e_DEs"] * 10

ra = np.array(ra)
dec = np.array(dec)
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

# No proprt motion and parallax estimate available
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)

# Other information
eph = ["DE200"] * psr_num
ref = ["wmz+01"] * psr_num
comment = [""] * psr_num

wmz01_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

# Remove PSRs without RA or dec measurements
mask = (wmz01_tab["ra"] != 0 )&(wmz01_tab["dec"] != 0)
wmz01_tab = wmz01_tab[mask]

## Paper 10: [The Parkes multi-beam pulsar survey - I. Observing and data analysis systems, discovery and timing of 100 pulsars](https://ui.adsabs.harvard.edu/abs/2001MNRAS.328...17M/abstract) 

by Manchester, R. N. ; Lyne, A. G. ; Camilo, F. ; Bell, J. F. ; Kaspi, V. M. ; D'Amico, N. ; McKay, N. P. F. ; Crawford, F. ; Stairs, I. H. ; Possenti, A. ; Kramer, M. ; Sheppard, D. C., where Table 4-5 can be downloaded from [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/328/17).

The timing analysis uses **DE200**.

In [14]:
mlc01_tab4 = Table.read("../data/J_MNRAS_328_17_table4.dat.fits.gz")
mlc01_tab5 = Table.read("../data/J_MNRAS_328_17_table5.dat.fits.gz")

mlc01_tab4.keep_columns([
    "PSRJ", "RAh", "RAm", "RAs", "e_RAs", "DE-", "DEd", "DEm", "DEs", "e_DEs"
])
mlc01_tab5.keep_columns(["PSRJ", "Epoch"])

psr_num = len(mlc01_tab4)

# Bytes to str
psr_name = ["J" + psr_namei.strip() for psr_namei in mlc01_tab4["PSRJ"]]

# Position epoch
pos_epoch = [float(pos_epochi) for pos_epochi in mlc01_tab5["Epoch"]]

dec_sign = np.where(mlc01_tab4["DE-"] == "-", -1, 1)

ra = (mlc01_tab4["RAh"] + mlc01_tab4["RAm"] / 60 + mlc01_tab4["RAs"] / 3600) * 15
dec = dec_sign * (mlc01_tab4["DEd"] + mlc01_tab4["DEm"] / 60 +
                  mlc01_tab4["DEs"] / 3600)

# mas
ra_err = mlc01_tab4["e_RAs"] * 15e3 * np.cos(np.deg2rad(dec))
dec_err = mlc01_tab4["e_DEs"] * 1e3

ra = np.array(ra)
dec = np.array(dec)
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

# No proprt motion and parallax estimate available
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)

# Other information
eph = ["DE200"] * psr_num
ref = ["mlc+01"] * psr_num
comment = [""] * psr_num

mlc01_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])


## Paper 11: [The Parkes Multibeam Pulsar Survey - III. Young pulsars and the discovery and timing of 200 pulsars](https://ui.adsabs.harvard.edu/abs/2003MNRAS.342.1299K/abstract) 

by Kramer, M. ; Bell, J. F. ; Manchester, R. N. ; Lyne, A. G. ; Camilo, F. ; Stairs, I. H. ; D'Amico, N. ; Kaspi, V. M. ; Hobbs, G. ; Morris, D. J. ; Crawford, F. ; Possenti, A. ; Joshi, B. C. ; McLaughlin, M. A. ; Lorimer, D. R. ; Faulkner, A. J., where Table 1-2 can be downloaded from [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/342/1299).

The timing analysis uses **DE200**.

In [15]:
kbm03_tab1 = Table.read("../data/J_MNRAS_342_1299_table1.dat.fits.gz")
kbm03_tab2 = Table.read("../data/J_MNRAS_342_1299_table2.dat.fits.gz")

kbm03_tab1.keep_columns([
    "PSRJ", "RAh", "RAm", "RAs", "e_RAs", "DE-", "DEd", "DEm", "DEs", "e_DEs"
])
kbm03_tab2.keep_columns(["PSRJ", "ObsMJD"])

psr_num = len(kbm03_tab1)

# Bytes to str
psr_name = ["J" + psr_namei.strip() for psr_namei in kbm03_tab1["PSRJ"]]

# Position epoch
pos_epoch = [float(pos_epochi) for pos_epochi in kbm03_tab2["ObsMJD"]]

dec_sign = np.where(kbm03_tab1["DE-"] == "-", -1, 1)

ra = (kbm03_tab1["RAh"] + kbm03_tab1["RAm"] / 60 + kbm03_tab1["RAs"] / 3600) * 15
dec = dec_sign * (kbm03_tab1["DEd"] + kbm03_tab1["DEm"] / 60 +
                  kbm03_tab1["DEs"] / 3600)

# mas
ra_err = kbm03_tab1["e_RAs"] * 15e3 * np.cos(np.deg2rad(dec))
dec_err = kbm03_tab1["e_DEs"] * 1e3

ra = np.array(ra)
dec = np.array(dec)
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

# No proprt motion and parallax estimate available
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)

# Other information
eph = ["DE200"] * psr_num
ref = ["kmb+03"] * psr_num
comment = [""] * psr_num

kbm03_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 12: [Long-term timing observations of 374 pulsars](https://ui.adsabs.harvard.edu/abs/2004MNRAS.353.1311H/abstract) 

by HOBBS G., LYNE A.G., KRAMER M., MARTIN C.E. and JORDAN C., where Table 2 and 6 can be downloaded from [CDS](https://vizier.cds.unistra.fr/viz-bin/VizieR?-source=J/MNRAS/353/1311).

The timing analysis uses **DE200**.

In [16]:
hlk04_tab2 = Table.read("../data/J_MNRAS_353_1311_table2.dat.fits.gz")

# pmRA, pmDE=0?
# hlk04_tab6 = Table.read("../data/J_MNRAS_353_1311_table6.dat.fits.gz")
#
hlk04_tab6 = Table.read("../data/J_MNRAS_353_1311_table6.dat",
                        format="ascii.fixed_width_no_header",
                        data_start=8, data_end=382,
                        delimiter="|",
                        names=[
                            "PSRJ", "f_PSRJ", "PSRB", "ELON", "e_ELON", "ELAT",
                            "e_ELAT", "pmELON", "e_pmELON", "pmELAT",
                            "e_pmELAT", "pmRA", "e_pmRA", "pmDE", "e_pmDE",
                            "NTOA", "Epoch", "Span", "tRMS"
                        ])


# Convert the coordinate from "HHMMSS DDMMSS" to values in degree.
# Bytes to str
psr_name = [ psr_namei.strip() for psr_namei in hlk04_tab6["PSRJ"] ]

dec_sign = np.where(hlk04_tab2["DE-"] == "-", -1, 1)

ra = (hlk04_tab2["RAh"] + hlk04_tab2["RAm"] / 60 +
      hlk04_tab2["RAs"] / 3600) * 15
dec = dec_sign * (hlk04_tab2["DEd"] + hlk04_tab2["DEm"] / 60 +
                  hlk04_tab2["DEs"] / 3600)

# mas
ra_err = hlk04_tab2["e_RAs"] * 15 * np.cos(np.deg2rad(dec)) * 1e3
dec_err = hlk04_tab2["e_DEs"] * 1e3

ra = np.array(ra)
dec = np.array(dec)
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

# Filling masked column
hlk04_tab6["pmRA"] = hlk04_tab6["pmRA"].filled(0)
hlk04_tab6["pmDE"] = hlk04_tab6["pmDE"].filled(0)
hlk04_tab6["e_pmRA"] = hlk04_tab6["e_pmRA"].filled(0)
hlk04_tab6["e_pmDE"] = hlk04_tab6["e_pmDE"].filled(0)

# No parallax estimate available
plx = np.zeros_like(ra)
plx_err = np.zeros_like(ra)

# Other information
eph = ["DE200"] * len(ra)
ref = ["hlk+04"] * len(ra)
comment = [""] * len(ra)

# Join two tables.

hlk04_tab = Table([
    psr_name,
    hlk04_tab2["Epoch"],
    eph, 
    ra,
    dec,
    hlk04_tab6["pmRA"],
    hlk04_tab6["pmDE"],
    plx,
    ra_err,
    dec_err,
    hlk04_tab6["e_pmRA"],
    hlk04_tab6["e_pmDE"],
    plx_err,
    ref,
    comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra", "pmdec",
                      "plx", "ra_err", "dec_err", "pmra_err", "pmdec_err",
                      "plx_err", "ref", "comment"
                  ])

## Paper 13: [The Parkes multibeam pulsar survey - IV. Discovery of 180 pulsars and parameters for 281 previously known pulsars](https://ui.adsabs.harvard.edu/abs/2004MNRAS.352.1439H/abstract) 

by Hobbs, G. ; Faulkner, A. ; Stairs, I. H. ; Camilo, F. ; Manchester, R. N. ; Lyne, A. G. ; Kramer, M. ; D'Amico, N. ; Kaspi, V. M. ; Possenti, A. ; McLaughlin, M. A. ; Lorimer, D. R. ; Burgay, M. ; Joshi, B. C. ; Crawford, F., where Table 1-2 can be downloaded from [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/352/1439).

The timing analysis uses **DE???(I am asking it)**.

In [17]:
hfs04_tab1 = Table.read("../data/J_MNRAS_352_1439_table1.dat",
                        format="ascii.fixed_width_no_header",
                        data_start=8,
                        data_end=188,
                        delimiter="|",
                        names=[
                            "PSRJ", "f_PSRJ", "RA", "e_RAs", "DE", "e_DEs",
                            "GLON_GLAT", "Beam", "Dist", "S/N", "S1.4",
                            "e_S1.4", "W50", "W10"
                        ],
                        include_names=[
                            "PSRJ",
                            "RA",
                            "e_RAs",
                            "DE",
                            "e_DEs",
                        ])

hfs04_tab2 = Table.read("../data/J_MNRAS_352_1439_table2.dat",
                        format="ascii.fixed_width_no_header",
                        data_start=8,
                        data_end=188,
                        delimiter="|",
                        names=[
                            "PSRJ", "f_PSRJ", "n_PSRJ", "Per", "e_Per",
                            "dP/dt", "e_dP/dt", "MJD", "NTOA", "MJD1_MJD2",
                            "Res", "DM", "e_DM"
                        ],
                        include_names=[
                            "MJD",
                        ])

psr_num = len(hfs04_tab1)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["hfs+04"] * psr_num
comment = [""] * psr_num

# Bytes to str
psr_name = [psr_namei.strip() for psr_namei in hfs04_tab1["PSRJ"]]

for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str3(
        hfs04_tab1["RA"][i], hfs04_tab1["DE"][i],
        hfs04_tab1["e_RAs"][i], hfs04_tab1["e_DEs"][i])


hfs04_tab = Table([
    psr_name, hfs04_tab2["MJD"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 14: [Timing measurements and proper motions of 74 pulsars using the Nanshan radio telescope](https://ui.adsabs.harvard.edu/abs/2005MNRAS.362.1189Z/abstract) 

by Zou, W. Z. ; Hobbs, G. ; Wang, N. ; Manchester, R. N. ; Wu, X. J. ; Wang, H. X., where Table 1 can be downloaded from [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/362/1189).

The timing analysis uses **DE405**.

In [18]:
zhw05_tab = Table.read("../data/J_MNRAS_362_1189_table1.dat.fits.gz")

psr_num = len(zhw05_tab)

# Bytes to str
psr_name = ["J" + psr_namei.strip() for psr_namei in zhw05_tab["PSRJ"]]
pos_epoch = [ float(pos_epochi) for pos_epochi in zhw05_tab["Epoch"]]

dec_sign = np.where(zhw05_tab["DE-"] == "-", -1, 1)

ra = (zhw05_tab["RAh"] + zhw05_tab["RAm"] / 60 + zhw05_tab["RAs"] / 3600) * 15
dec = dec_sign * (zhw05_tab["DEd"] + zhw05_tab["DEm"] / 60 +
                  zhw05_tab["DEs"] / 3600)

# mas
ra_err = zhw05_tab["e_RAs"] * 15 * np.cos(np.deg2rad(dec)) * 1e3
dec_err = zhw05_tab["e_DEs"] * 1e3

ra = np.array(ra)
dec = np.array(dec)
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

# No proprt motion and parallax estimate available
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)

# Other information
eph = ["DE405"] * psr_num
ref = ["zhw+05"] * psr_num
comment = [""] * psr_num

zhw05_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

## Paper 15: [The Parkes Multibeam Pulsar Survey - VI. Discovery and timing of 142 pulsars and a Galactic population analysis.](https://ui.adsabs.harvard.edu/abs/2006MNRAS.372..777L/abstract) 

by LORIMER D.R., FAULKNER A.J., LYNE A.G., MANCHESTER R.N., KRAMER M., McLAUGHLIN M.A., HOBBS G., POSSENTI A., STAIRS I.H., CAMILO F., BURGAY M., D'AMICO N., CORONGIU A. and CRAWFORD F., where Table 1 can be downloaded from [CDS](https://vizier.cds.unistra.fr/viz-bin/VizieR?-source=J/MNRAS/372/777).

The timing analysis uses **DE200???** (I assume that they used the same DE ephemeris in the whole survey).

In [19]:
lfl06_tab1 = Table.read("../data/J_MNRAS_372_777_table1.dat.fits.gz")
lfl06_tab2 = Table.read("../data/J_MNRAS_372_777_table2.dat.fits.gz")

psr_num = len(lfl06_tab1)

# Bytes to str
psr_name = ["J" + psr_namei.strip() for psr_namei in lfl06_tab1["PSRJ"]]
pos_epoch = [ pos_epochi for pos_epochi in lfl06_tab2["Epoch"]]

dec_sign = np.where(lfl06_tab1["DE-"] == "-", -1, 1)

ra = (lfl06_tab1["RAh"] + lfl06_tab1["RAm"] / 60 + lfl06_tab1["RAs"] / 3600) * 15
dec = dec_sign * (lfl06_tab1["DEd"] + lfl06_tab1["DEm"] / 60 +
                  lfl06_tab1["DEs"] / 3600)

# mas
ra_err = lfl06_tab1["e_RAs"] * 15 * np.cos(np.deg2rad(dec)) * 1e3
dec_err = lfl06_tab1["e_DEs"] * 1e3

ra = np.array(ra)
dec = np.array(dec)
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

# No proprt motion and parallax estimate available
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)

# Other information
eph = ["DE200"] * psr_num
ref = ["lfl+06"] * psr_num
comment = [""] * psr_num

lfl06_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 16: [High-precision baseband timing of 15 millisecond pulsars](https://ui.adsabs.harvard.edu/abs/2006MNRAS.369.1502H/abstract) 

by Hotan, A. W. ; Bailes, M. ; Ord, S. M., where Table 2-4 can be extracted directly from the paper.

The timing analysis uses **DE405**.

In [20]:
hbo06_tab = Table.read("../data/hbo06_table.dat", format="ascii.csv")

psr_num = len(hbo06_tab)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE405"] * psr_num
ref = ["hbo06"] * psr_num
comment = [""] * psr_num

# Translate B1950 name into J2000 name
psr_name = []
for psr_namei in hbo06_tab["psr_name"]:
    if psr_namei in psr_name_map.keys():
        psr_name.append(psr_name_map[psr_namei])
    else:
        psr_name.append(psr_namei)


for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        hbo06_tab["ra_str"][i], hbo06_tab["dec_str"][i])

    pmra[i], pmra_err[i] = parse_pmp_str2(hbo06_tab["pmra_str"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str2(hbo06_tab["pmdec_str"][i])

hbo06_tab = Table([
    psr_name, hbo06_tab["pos_epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 17: [Discovery of 14 Radio Pulsars in a Survey of the Magellanic Clouds](https://ui.adsabs.harvard.edu/abs/2006ApJ...649..235M/abstract) 

by Manchester, R. N. ; Fan, G. ; Lyne, A. G. ; Kaspi, V. M. ; Crawford, F., where Table 1-2 can be extracted directly from the paper via [OCR](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE200**.

In [21]:
mfl06_tab1 = Table.read(
    "../data/mfl+06_table1.dat",
    format="ascii.fixed_width_no_header",
    data_start=2,
    col_starts=[2, 25, 41],
    col_ends=[12, 40, 57],
    names=["psr_jname", "ra_str", "dec_str", ])

mfl06_tab2 = Table.read(
    "../data/mfl+06_table2.dat",
    format="ascii.fixed_width_no_header",
    data_start=3,
    col_starts=[51],
    col_ends=[58],
    names=["pos_epoch"])

psr_num = len(mfl06_tab1)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE200"] * psr_num
ref = ["mfl+06"] * psr_num
comment = [""] * psr_num

# psr_name = ["J" + psr_namei.strip() for psr_namei in mfl06_tab1["psr_jname"]]

for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        mfl06_tab1["ra_str"][i], mfl06_tab1["dec_str"][i])

mfl06_tab = Table([
    mfl06_tab1["psr_jname"], mfl06_tab2["pos_epoch"], eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 18: [Timing stability of millisecond pulsars and prospects for gravitational-wave detection](https://ui.adsabs.harvard.edu/abs/2009MNRAS.400..951V/abstract) 

by Verbiest, J. P. W. ; Bailes, M. ; Coles, W. A. ; Hobbs, G. B. ; van Straten, W. ; Champion, D. J. ; Jenet, F. A. ; Manchester, R. N. ; Bhat, N. D. R. ; Sarkissian, J. M. ; Yardley, D. ; Burke-Spolaor, S. ; Hotan, A. W. ; You, X. P., where Table 3, 4 and 5 can be used and data are extracted directly from the paper via [Table OCR tool](https://products.aspose.app/ocr/en/table-ocr).

The timing analysis uses **DE405**. 

In [22]:
vbc09_tab = Table.read("../data/vbc+09_table.dat", format="ascii.csv")

psr_num = len(vbc09_tab)

ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE405"] * psr_num
ref = ["vbc+09"] * psr_num
comment = [""] * psr_num

pos_epoch = [ float(pos_epochi) for pos_epochi in vbc09_tab["pos_epoch"]]

for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        vbc09_tab["ra_str"][i], vbc09_tab["dec_str"][i])

    pmra[i], pmra_err[i] = parse_pmp_str2(vbc09_tab["pmra_str"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str2(vbc09_tab["pmdec_str"][i])
    plx[i], plx_err[i] = parse_pmp_str2(vbc09_tab["plx_str"][i])

vbc09_tab = Table([
    vbc09_tab["psr_name"], pos_epoch, eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

## Paper 19: [Precise γ-ray Timing and Radio Observations of 17 Fermi γ-ray Pulsars](https://ui.adsabs.harvard.edu/abs/2011ApJS..194...17R/abstract) 

by Ray, P. S. ; Kerr, M. ; Parent, D. ; Abdo, A. A. ; Guillemot, L. ; Ransom, S. M. ; Rea, N. ; Wolff, M. T. ; Makeev, A. ; Roberts, M. S. E. ; Camilo, F. ; Dormody, M. ; Freire, P. C. C. ; Grove, J. E. ; Gwon, C. ; Harding, A. K. ; Johnston, S. ; Keith, M. ; Kramer, M. ; Michelson, P. F. ; ..., where Table 2-18 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).


The timing analysis uses **DE405**. 

And there is no measurement for the proper motion and parallax given in the paper.

In [23]:
rkp11_tab = Table.read("../data/rkp+11_table.dat", format="ascii")

psr_num = len(rkp11_tab)

ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE405"] * psr_num
ref = ["rkp+11"] * psr_num
comment = [""] * psr_num


for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str3(
        rkp11_tab["ra_str"][i],
        rkp11_tab["dec_str"][i],
        rkp11_tab["ra_err"][i],
        rkp11_tab["dec_err"][i])


rkp11_tab = Table([
    rkp11_tab["psr_name"], rkp11_tab["pos_epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 20: [Limits on the Stochastic Gravitational Wave Background from the North American Nanohertz Observatory for Gravitational Waves](https://ui.adsabs.harvard.edu/abs/2013ApJ...762...94D/abstract) 

by Stovall, K. ; Lynch, R. S. ; Ransom, S. M. ; Archibald, A. M. ; Banaszak, S. ; Biwer, C. M. ; Boyles, J. ; Dartez, L. P. ; Day, D. ; Ford, A. J. ; Flanigan, J. ; Garcia, A. ; Hessels, J. W. T. ; Hinojosa, J. ; Jenet, F. A. ; Kaplan, D. L. ; Karako-Argaman, C. ; Kaspi, V. M. ; Kondratiev, V. I. ; Leake, S. ; ..., where the timing solution can be downloaded from [NANOGrav website](https://data.nanograv.org/).


The timing analysis uses **DE405**. 

In [24]:
dfg13_tab = Table.read("../data/nanograv_5yr.dat", format="ascii")

psr_num = len(dfg13_tab)

ref = ["dfg+13"] * psr_num
comment = ["NANOGrav-5y"] * psr_num

ref = Column(ref, name="ref")
comment = Column(comment, name="comment")

for i in range(psr_num):
    psr_namei = dfg13_tab["psr_name"][i]

    if len(psr_namei) == 7:
        pser_namei = "B" + psr_namei
    elif len(psr_namei) == 9:
        pser_namei = "J" + psr_namei
    else:
        print(psr_namei)
        sys.exit()

    if psr_namei in psr_name_map.keys():
        psr_namei = psr_name_map[psr_namei]
        
    dfg13_tab["psr_name"][i] = psr_namei
        
dfg13_tab.add_columns([ref, comment])

## Paper 21: [The Green Bank Northern Celestial Cap Pulsar Survey. I. Survey Description, Data Analysis, and Initial Results](https://ui.adsabs.harvard.edu/abs/2014ApJ...791...67S/abstract) 

by Stovall, K. ; Lynch, R. S. ; Ransom, S. M. ; Archibald, A. M. ; Banaszak, S. ; Biwer, C. M. ; Boyles, J. ; Dartez, L. P. ; Day, D. ; Ford, A. J. ; Flanigan, J. ; Garcia, A. ; Hessels, J. W. T. ; Hinojosa, J. ; Jenet, F. A. ; Kaplan, D. L. ; Karako-Argaman, C. ; Kaspi, V. M. ; Kondratiev, V. I. ; Leake, S. ; ..., where Table 2-4 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).


The timing analysis uses **DE421**. 

And there is no measurement for the parallax given in the paper.

In [25]:
slr14_tab = Table.read("../data/slr+14_table.dat", format="ascii.csv")

psr_num = len(slr14_tab)

ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE421"] * psr_num
ref = ["slr+14"] * psr_num
comment = [""] * psr_num


for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        slr14_tab["ra_str"][i], slr14_tab["dec_str"][i])

    pmra[i], pmra_err[i] = parse_pmp_str2(slr14_tab["pmra_str"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str2(slr14_tab["pmdec_str"][i])

slr14_tab = Table([
    slr14_tab["psr_name"], slr14_tab["pos_epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 22: [Proper motions of 15 pulsars: a comparison between Bayesian and frequentist algorithms](https://ui.adsabs.harvard.edu/abs/2016MNRAS.460.4011L/abstract) 

by Li, L. ; Wang, N. ; Yuan, J. P. ; Wang, J. B. ; Hobbs, G. ; Lentati, L. ; Manchester, R. N., where Table 1 and 2 can be used and data are extracted via [Table OCR tool](https://products.aspose.app/ocr/table-ocr).


The timing analysis uses **DE421**. 

And there is no measurement for the parallax given in the paper.

In [26]:
lwy16_tab1 = Table.read("../data/lwy+16_table1.dat",
                        format="ascii.no_header",
                        data_start=2,
                        names=[
                            "psrj_name", "psrb_name", "epoch_f", "ra_f",
                            "dec_f", "epoch_c", "ra_c", "dec_c", "ref"
                        ])

lwy16_tab2 = Table.read("../data/lwy+16_table2.dat",
                        format="ascii.no_header",
                        data_start=2,
                        names=[
                            "psrj_name", "psrb_name", "pmra_f", "pmdec_f",
                            "pmra_b", "pmdec_b", "pmra_cp", "pmdec_cp",
                            "pmra_t", "pmdec_t", "pmra_i", "pmdec_i", "ref"
                        ])

psr_num = len(lwy16_tab1)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE421"] * psr_num
ref = ["lwy+16"] * psr_num
comment = [""] * psr_num

pos_epoch = [ float(pos_epochi) for pos_epochi in lwy16_tab1["epoch_f"]]

for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        lwy16_tab1["ra_f"][i], lwy16_tab1["dec_f"][i])

    pmra[i], pmra_err[i] = parse_pmp_str(lwy16_tab2["pmra_f"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str(lwy16_tab2["pmdec_f"][i])

lwy16_tab = Table([
    lwy16_tab1["psrj_name"], pos_epoch, eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

## Paper 23: [High-precision timing of 42 millisecond pulsars with the European Pulsar Timing Array](https://ui.adsabs.harvard.edu/abs/2016MNRAS.458.3341D/abstract) 

by Desvignes, G. ; Caballero, R. N. search by orcid ; Lentati, L. ; Verbiest, J. P. W. ; Champion, D. J. ; Stappers, B. W. ; Janssen, G. H. ; Lazarus, P. ; Osłowski, S. ; Babak, S. ; Bassa, C. G. ; Brem, P. ; Burgay, M. ; Cognard, I. ; Gair, J. R. ; Graikou, E. ; Guillemot, L. ; Hessels, J. W. T. ; Jessner, A. ; Jordan, C. ; et al., where Table 2 - 12 can be used and data are extracted via [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/458/3341).

The timing analysis uses **DE421**. 

In [27]:
dcl16_tab = Table.read("../data/J_MNRAS_458_3341_tab2-12.dat",
                       format="ascii.fixed_width_no_header",
                       data_start=7,
                       data_end=49,
                       delimiter="|")

dcl16_tab.keep_columns([
    "col1", "col5", "col6", "col7", "col8", "col9", "col10", "col11", "col12",
    "col13", "col18", "col19"
])

psr_num = len(dcl16_tab)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)

pmra = dcl16_tab["col10"]
pmdec = dcl16_tab["col12"]
pmra_err = dcl16_tab["col11"]
pmdec_err = dcl16_tab["col13"]

plx_err = dcl16_tab["col19"].filled(0)
eph = ["DE421"] * psr_num
ref = ["dcl+16"] * psr_num
comment = ["EPTA"] * psr_num

pos_epoch = [float(pos_epochi) for pos_epochi in dcl16_tab["col5"]]

plx = np.zeros(psr_num)

for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str3(
        dcl16_tab["col6"][i], dcl16_tab["col8"][i], dcl16_tab["col7"][i],
        dcl16_tab["col9"][i])

    if dcl16_tab["col18"][i] != "---":
        plx[i] = float(dcl16_tab["col18"][i])

dcl16_tab = Table([
    dcl16_tab["col1"], pos_epoch, eph, ra, dec, pmra, pmdec,
    plx, ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 24: [The NANOGrav Nine-year Data Set: Astrometric Measurements of 37 Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/2016ApJ...818...92M/abstract) 

by Matthews, Allison M. ; Nice, David J. ; Fonseca, Emmanuel ; Arzoumanian, Zaven ; Crowter, Kathryn ; Demorest, Paul B. ; Dolch, Timothy ; Ellis, Justin A. ; Ferdman, Robert D. ; Gonzalez, Marjorie E. ; Jones, Glenn ; Jones, Megan L. ; Lam, Michael T. ; Levin, Lina ; McLaughlin, Maura A. ; Pennucci, Timothy T. ; Ransom, Scott M. ; Stairs, Ingrid H. ; Stovall, Kevin ; Swiggum, Joseph K. ; ..., where Table 1-3 can be used and data are extracted via [ApJ webpage](https://iopscience.iop.org/article/10.3847/0004-637X/818/1/92).

The timing analysis uses **DE421**. 

In [28]:
mnf16_tab1 = Table.read(
    "../data/apj522511t1_ascii.txt",
    format="ascii.no_header",
    data_start=4,
    data_end=41,
    names=["psr_name", "ecl_lon", "ecl_lat", "ra_str", "dec_str", "pos_epoch"])

mnf16_tab2 = Table.read("../data/apj522511t2_ascii.txt",
                        format="ascii.no_header",
                        data_start=5,
                        data_end=42,
                        names=[
                            "psr_name", "dt", "pmecl_lon", "pmecl_lat",
                            "pmra_str", "pmdec_str", "pmgal_lon", "pmgal_lat",
                            "pmra_str1", "pmdec_str1", "ref"
                        ])

mnf16_tab3 = Table.read("../data/apj522511t3_ascii.txt",
                        format="ascii.no_header",
                        data_start=5,
                        data_end=42,
                        names=["psr_name", "plx_str", "plx_str1", "ref"])

psr_num = len(mnf16_tab1)
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE421"] * psr_num
ref = ["mnf+16"] * psr_num
comment = ["NANOGrav-9yr"] * psr_num
psr_name = []

for i in range(psr_num):
    # Translate B1950 name into J2000 name
    psr_namei = mnf16_tab1["psr_name"][i]
    if psr_namei in psr_name_map.keys():
        psr_name.append(psr_name_map[psr_namei])
    else:
        psr_name.append(psr_namei)

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        mnf16_tab1["ra_str"][i], mnf16_tab1["dec_str"][i])

    if mnf16_tab2["pmra_str"][i] != "cdots":
        pmra[i], pmra_err[i] = parse_pmp_str2(mnf16_tab2["pmra_str"][i])

    if mnf16_tab2["pmdec_str"][i] != "cdots":
        pmdec[i], pmdec_err[i] = parse_pmp_str2(mnf16_tab2["pmdec_str"][i])

    if mnf16_tab3["plx_str"][i] != "cdots":
        plx[i], plx_err[i] = parse_pmp_str2(mnf16_tab3["plx_str"][i])

mnf16_tab = Table([
    psr_name, mnf16_tab1["pos_epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 25: [The International Pulsar Timing Array: First data release](https://ui.adsabs.harvard.edu/abs/2018ApJS..235...37A/abstract) 

by Verbiest, J. P. W. ; Lentati, L. ; Hobbs, G. ; van Haasteren, R. ; Demorest, P. B. ; Janssen, G. H. ; Wang, J. -B. ; Desvignes, G. ; Caballero, R. N. ; Keith, M. J. ; Champion, D. J. ; Arzoumanian, Z. ; Babak, S. ; Bassa, C. G. ; Bhat, N. D. R. ; Brazier, A. ; Brem, P. ; Burgay, M. ; Burke-Spolaor, S. ; Chamberlin, S. J. ; ..., where the timing solution can be downloaded from [Gitlab](https://gitlab.com/IPTA/DR1).

The timing analysis uses **DE421**. 

In [29]:
vlh16_tab = Table.read("../data/ipta-dr1.dat", format="ascii")

psr_num = len(vlh16_tab)

ref = ["vlh+16"] * psr_num
comment = ["IPTA-DR1"] * psr_num

ref = Column(ref, name="ref")
comment = Column(comment, name="comment")

vlh16_tab.add_columns([ref, comment])

## Paper 26: [Timing analysis for 20 millisecond pulsars in the Parkes Pulsar Timing Array](https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.1751R/abstract) 

by Reardon, D. J. ; Hobbs, G. ; Coles, W. ; Levin, Y. ; Keith, M. J. ; Bailes, M. ; Bhat, N. D. R. ; Burke-Spolaor, S. ; Dai, S. ; Kerr, M. ; Lasky, P. D. ; Manchester, R. N. ; Osłowski, S. ; Ravi, V. ; Shannon, R. M. ; van Straten, W. ; Toomey, L. ; Wang, J. ; Wen, L. ; You, X. P. ; ..., where Table A1 - A4 can be downloaded from [CSIRO Data Access Portal](http://dx.doi.org/10.4225/08/561EFD72D0409).

The timing analysis uses **DE421**. 

In [30]:
rhc16_tab = Table.read("../data/ppta_dr1e.dat", format="ascii")

psr_num = len(rhc16_tab)

ref = ["rhc+16"] * psr_num
comment = ["PPTA-DR1e"] * psr_num

ref = Column(ref, name="ref")
comment = Column(comment, name="comment")

rhc16_tab.add_columns([ref, comment])

## Paper 27: [The NANOGrav 11-year Data Set: High-precision Timing of 45 Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/2018ApJS..235...37A/abstract) 

by Arzoumanian, Zaven ; Brazier, Adam ; Burke-Spolaor, Sarah et al., where Table 4 and 5 can be used and data are extracted via [ApJ webpage](https://iopscience.iop.org/article/10.3847/1538-4365/aab5b0).

The timing analysis uses **DE436**. 

In [31]:
# Position and proper motion
abb18_tab4 = Table.read(
    "../data/apjsaab5b0t4_ascii.txt",
    format="ascii.no_header",
    data_start=4,
    data_end=49,
    names=["psr_name", "ra_str", "dec_str", "pmra_str", "pmdec_str", "epoch"])

# parallax
abb18_tab5 = Table.read("../data/apjsaab5b0t5_ascii.txt",
                        format="ascii.no_header",
                        comment="#",
                        delimiter="\t",
                        names=[
                            "psr_name", "plx_str", "dist", "plx_c", "plx_ref",
                            "note1", "note2", "note3"
                        ],
                        include_names=["psr_name", "plx_str"])

# Combine them together
abb18_tab = join(abb18_tab4, abb18_tab5, keys="psr_name")

psr_num = len(abb18_tab)

ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE436"] * psr_num
ref = ["abb+18"] * psr_num
comment = ["NANOGrav-11yr"] * psr_num

# Translate B1950 name into J2000 name
psr_name = []
for psr_namei in abb18_tab["psr_name"]:
    if psr_namei in psr_name_map.keys():
        psr_name.append(psr_name_map[psr_namei])
    else:
        psr_name.append(psr_namei)

for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        abb18_tab["ra_str"][i], abb18_tab["dec_str"][i])

    pmra[i], pmra_err[i] = parse_pmp_str2(abb18_tab["pmra_str"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str2(abb18_tab["pmdec_str"][i])
    plx[i], plx_err[i] = parse_pmp_str2(abb18_tab["plx_str"][i])

abb18_tab = Table([
    psr_name, abb18_tab["epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

## Paper 28: [The UTMOST pulsar timing programme I: Overview and first results](https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.3691J/abstract) 

by Jankowski, F.; Bailes, M.; van Straten, W.; Keane, E. F.; Flynn, C.; Barr, E. D.; Bateman, T.; Bhandari, S.; Caleb, M.; Campbell-Wilson, D.; Farah, W.; Green, A. J.; Hunstead, R. W.; Jameson, A.; Osłowski, S.; Parthasarathy, A.; Rosado, P. A.; Venkatraman Krishnan, V., where Table 3, B1 and B22 can be downloaded from [CDS](https://cdsarc.cds.unistra.fr/viz-bin/cat/J/MNRAS/484/3691).

The timing analysis uses **DE430**. 

And there is no measurement for the parallax given in the paper.

*The period, position, and DM determination epoch is set to **MJD 57600** for all pulsars.*

In [32]:
# Proper motion
jbs19_tab3 = Table.read(
    "../data/J_MNRAS_484_3691_table3.dat",
    format="ascii.no_header",
    delimiter="|",
    data_start=9,
    data_end=69,
    names=[
        "indx", "psr_name", "note1", "pmra", "pmra_err", "pmdec", "pmdec_err",
        "pmra1", "pmra_err1", "note2", "pmdec1", "pmdec_err1", "note3", "dt",
        "#pos", "ref", "d", "d_err", "Vt", "Vt_err"
    ],
    include_names=["psr_name", "pmra", "pmra_err", "pmdec", "pmdec_err"])

# Position for binary PSR
jbs19_tabb1 = Table.read("../data/J_MNRAS_484_3691_tableb1.dat.fits.gz")

# Bytes to str
psr_name = [psr_namei.strip() for psr_namei in jbs19_tabb1["PSRJ"]]

# RA and decl.
dec_sign = np.where(jbs19_tabb1["DE-"] == "-", -1, 1)
ra = (jbs19_tabb1["RAh"] + jbs19_tabb1["RAm"] / 60 +
      jbs19_tabb1["RAs"] / 3600) * 15
dec = dec_sign * (jbs19_tabb1["DEd"] + jbs19_tabb1["DEm"] / 60 +
                  jbs19_tabb1["DEs"] / 3600)
ra = np.array(ra)
dec = np.array(dec)

# mas
ra_err = jbs19_tabb1["e_RAs"] * 15 * np.cos(np.deg2rad(dec)) * 1e3
dec_err = jbs19_tabb1["e_DEs"] * 1e3
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

jbs19_tabb1 = Table([psr_name, ra, dec, ra_err, dec_err],
                    names=[
                        "psr_name",
                        "ra",
                        "dec",
                        "ra_err",
                        "dec_err",
                    ])

# Position for binary PSR
jbs19_tabb2 = Table.read("../data/J_MNRAS_484_3691_tableb2.dat.fits.gz")

# Bytes to str
psr_name = [psr_namei.strip() for psr_namei in jbs19_tabb2["PSRJ"]]

# RA and decl.
dec_sign = np.where(jbs19_tabb2["DE-"] == "-", -1, 1)
ra = (jbs19_tabb2["RAh"] + jbs19_tabb2["RAm"] / 60 +
      jbs19_tabb2["RAs"] / 3600) * 15
dec = dec_sign * (jbs19_tabb2["DEd"] + jbs19_tabb2["DEm"] / 60 +
                  jbs19_tabb2["DEs"] / 3600)
ra = np.array(ra)
dec = np.array(dec)

# mas
ra_err = jbs19_tabb2["e_RAs"] * 15 * np.cos(np.deg2rad(dec)) * 1e3
dec_err = jbs19_tabb2["e_DEs"] * 1e3
ra_err = np.array(ra_err)
dec_err = np.array(dec_err)

jbs19_tabb2 = Table([psr_name, ra, dec, ra_err, dec_err],
                    names=[
                        "psr_name",
                        "ra",
                        "dec",
                        "ra_err",
                        "dec_err",
                    ])

jbs19_tab = vstack((jbs19_tabb1, jbs19_tabb2))
jbs19_tab = join(jbs19_tab, jbs19_tab3, keys="psr_name", join_type="left")

# Filling masked column
jbs19_tab["pmra"] = jbs19_tab["pmra"].filled(0)
jbs19_tab["pmdec"] = jbs19_tab["pmdec"].filled(0)
jbs19_tab["pmra_err"] = jbs19_tab["pmra_err"].filled(0)
jbs19_tab["pmdec_err"] = jbs19_tab["pmdec_err"].filled(0)

psr_num = len(jbs19_tab)
pos_epoch = np.ones(psr_num) * 57600
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE430"] * psr_num
ref = ["jbs+19"] * psr_num
comment = ["UTMOST-I"] * psr_num

jbs19_tab = Table([
    jbs19_tab["psr_name"], pos_epoch, eph, jbs19_tab["ra"], jbs19_tab["dec"],
    jbs19_tab["pmra"], jbs19_tab["pmdec"], plx, jbs19_tab["ra_err"],
    jbs19_tab["dec_err"], jbs19_tab["pmra_err"], jbs19_tab["pmdec_err"],
    plx_err, ref, comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

## Paper 29: [Timing of young radio pulsars - I. Timing noise, periodic modulation, and proper motion](https://ui.adsabs.harvard.edu/abs/2019MNRAS.489.3810P/abstract) 

Parthasarathy, A. ; Shannon, R. M. ; Johnston, S. ; Lentati, L. ; Bailes, M. ; Dai, S. ; Kerr, M. ; Manchester, R. N. ; Osłowski, S. ; Sobey, C. ; van Straten, W. ; Weltevrede, P. ., where timing solution in table 1 can be downloaded via [OCR](https://products.aspose.app/ocr/en/table-ocr).

The timing analysis uses **DE???(I am asking it)**. 

In [33]:
psj19_tab = Table.read("../data/psj+19_table1.dat",
                       format="ascii.fixed_width_no_header",
                       data_start=3,
                       col_starts=[2, 21, 38, 55],
                       col_ends=[12, 37, 53, 60],
                       names=["psr_name", "ra_str", "dec_str", "pos_epoch"])


psr_num = len(psj19_tab)

ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE4??"] * psr_num
ref = ["psj+19"] * psr_num
comment = [""] * psr_num


for i in range(psr_num):
    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        psj19_tab["ra_str"][i], psj19_tab["dec_str"][i])

psj19_tab = Table([
    psj19_tab["psr_name"], psj19_tab["pos_epoch"], eph, ra, dec, pmra, pmdec, plx,
    ra_err, dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])



## Paper 30: [The International Pulsar Timing Array: second data release](https://ui.adsabs.harvard.edu/abs/2019MNRAS.490.4666P/abstract) 

by Perera, B. B. P. ; DeCesar, M. E. ; Demorest, P. B. ; Kerr, M. ; Lentati, L. ; Nice, D. J. ; Osłowski, S. ; Ransom, S. M. ; Keith, M. J. ; Arzoumanian, Z. ; Bailes, M. ; Baker, P. T. ; Bassa, C. G. ; Bhat, N. D. R. ; Brazier, A. ; Burgay, M. ; Burke-Spolaor, S. ; Caballero, R. N. ; Champion, D. J. ; Chatterjee, S. ; ..., where timing solution can be acessed via [Gitlab](https://gitlab.com/IPTA/DR2/tree/master/release).

The timing analysis uses **DE436**. 

In [34]:
pdd19_tab = Table.read("../data/ipta-dr2.dat", format="ascii")

psr_num = len(pdd19_tab)

ref = ["pdd+19"] * psr_num
comment = ["IPTA-DR2"] * psr_num

ref = Column(ref, name="ref")
comment = Column(comment, name="comment")

pdd19_tab.add_columns([ref, comment])

## Paper 31: [The UTMOST pulsar timing programme - II. Timing noise across the pulsar population](https://ui.adsabs.harvard.edu/abs/2020MNRAS.494..228L/abstract) 

by Lower, M. E.; Bailes, M.; Shannon, R. M.; Johnston, S.; Flynn, C.; Osłowski, S.; Gupta, V.; Farah, W.; Bateman, T.; Green, A. J.; Hunstead, R.; Jameson, A.; Jankowski, F.; Parthasarathy, A.; Price, D. C.; Sutherland, A.; Temby, D.; Venkatraman Krishnan, V., where timing solution can be downloaded from [github](https://github.com/Molonglo/TimingDataRelease1).

The data contained in the `.par` file are extracted using my code `extract-timing-data.py` in the same directory as this notebook.

In [35]:
lbs20_tab = Table.read("../data/utmost2-psr.dat", format="ascii", )

psr_num = len(lbs20_tab)
ref = ["lbs+20"] * psr_num
comment = ["UTMOST-II"] * psr_num

ref = Column(ref, name="ref")
comment = Column(comment, name="comment")
lbs20_tab.add_columns([ref, comment])

## Paper 32: [Results of 12 yr of Pulsar Timing at Nanshan. I.](https://ui.adsabs.harvard.edu/abs/2020ApJ...896..140D/abstract) 

by Dang, S. J. ; Yuan, J. P. ; Manchester, R. N. ; Li, L. ; Wang, N.; Wang, J. B. ; Hobbs, G. ; Liu, Z. Y. ; Kou, F. F., where Table 1 and 2 can be downloaded from [the ApJ webpage of this paper](https://iopscience.iop.org/article/10.3847/1538-4357/ab9082).

The timing analysis uses **DE421**. 

And there is no measurement for the parallax given in the paper.

In [36]:
dym20_tab1 = Table.read("../data/dym+20_table1.dat",
                        format="ascii.no_header",
                        data_start=4,
                        data_end=52,
                        names=[
                            "psrj_name", "psrb_name", "epoch_c", "ra_c",
                            "dec_c", "epoch_t", "ra_t", "dec_t", "ref"
                        ])

dym20_tab2 = Table.read("../data/dym+20_table2.dat",
                        format="ascii.no_header",
                        data_start=4,
                        data_end=52,
                        names=[
                            "psrj_name", "psrb_name", "pmra_t", "pmra_c",
                            "pmra_ct", "pmdec_t", "pmdec_c", "pmdec_ct", "ref"
                        ])

# Note that for some pulsars, 
# the declination and associated proper motion measurement is not given 
# (value of "cdots") in the Table 1 and 2, these pulsars are not used.

mask = (dym20_tab1["dec_t"] != "cdots")
dym20_tab1 = dym20_tab1[mask]
dym20_tab1.keep_columns(["psrj_name", "epoch_t", "ra_t", "dec_t"])

mask = (dym20_tab2["pmdec_t"] != "cdots")
dym20_tab2 = dym20_tab2[mask]
dym20_tab2.keep_columns(["psrj_name", "pmra_t", "pmdec_t"])
dym20_tab = join(dym20_tab1, dym20_tab2, keys="psrj_name")

# Convert the string into values.

psr_num = len(dym20_tab)
psr_name = []
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE421"] * psr_num
ref = ["dym+20"] * psr_num
comment = [""] * psr_num

pos_epoch = [ float(pos_epochi) for pos_epochi in dym20_tab["epoch_t"]]

for i in range(psr_num):
    psr_name.append(dym20_tab["psrj_name"][i][:10])

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        dym20_tab["ra_t"][i], dym20_tab["dec_t"][i])

    pmra[i], pmra_err[i] = parse_pmp_str(dym20_tab["pmra_t"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str(dym20_tab["pmdec_t"][i])

dym20_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
                  names=[
                      "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                      "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                      "pmdec_err", "plx_err", "ref", "comment"
                  ])

## Paper 33: [New timing measurement results of 16 pulsars](https://ui.adsabs.harvard.edu/abs/2020PASJ...72...70L/abstract) 

by Liu, Jie ; Yan, Zhen ; Shen, Zhi-Qiang ; Huang, Zhi-Peng ; Zhao, Ru-Shuang ; Wu, Ya-Jun ; Yuan, Jian-Ping ; Wu, Xin-Ji, where Table 1 can be extracted via [OCR](https://products.aspose.app/ocr/table-ocr).

The timing analysis uses **DE421**. 

In [37]:
lys20_tab = Table.read("../data/lys+20_table1.dat",
                       format="ascii.no_header",
                       data_start=2,
                       names=["psrb_name", "col2", "ra_str", "dec_str",
                              "pmra_str", "pmdec_str", "pos_epoch"],
                       exclude_names=["col2"])

psr_num = len(lys20_tab)
psr_name = []
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE421"] * psr_num
ref = ["lys+20"] * psr_num
comment = [""] * psr_num

pos_epoch = [float(pos_epochi) for pos_epochi in lys20_tab["pos_epoch"]]

for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        lys20_tab["ra_str"][i], lys20_tab["dec_str"][i])

    pmra[i], pmra_err[i] = parse_pmp_str(lys20_tab["pmra_str"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str(lys20_tab["pmdec_str"][i])

lys20_tab = Table([
    lys20_tab["psrb_name"], pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_bname", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])


## Paper 34: [The Parkes pulsar timing array second data release: timing analysis](https://ui.adsabs.harvard.edu/abs/2021MNRAS.507.2137R/abstract) 

by Reardon, D. J. ; Shannon, R. M. ; Cameron, A. D. ; Goncharov, B. ; Hobbs, G. B. ; Middleton, H. ; Shamohammadi, M. ; Thyagarajan, N. ; Bailes, M. ; Bhat, N. D. R. ; Dai, S. ; Kerr, M. ; Manchester, R. N. ; Russell, C. J. ; Spiewak, R. ; Wang, J. B. ; Zhu, X. J., where Table A1 - A4 can be downloaded from [CSIRO Data Access Portal](https://data.csiro.au/collection/csiro:50795v2).

The timing analysis uses **DE436**. 

In [38]:
rsc21_tab1 = Table.read("../data/ppta_dr2e.dat", format="ascii")
rsc21_tab2 = Table.read("../data/ppta_dr2e1.dat", format="ascii.csv")

psr_num = len(rsc21_tab2)

# Convert the string into values.
psr_num = len(rsc21_tab2)
psr_name = []
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
eph = ["DE436"] * psr_num
ref = ["rsc+21"] * psr_num
comment = ["PPTA-DR2e"] * psr_num


for i in range(psr_num):

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        rsc21_tab2["ra_str"][i], rsc21_tab2["dec_str"][i])

    pmra[i], pmra_err[i] = parse_pmp_str2(rsc21_tab2["pmra_str"][i])
    pmdec[i], pmdec_err[i] = parse_pmp_str2(rsc21_tab2["pmdec_str"][i])
    plx[i], plx_err[i] = parse_pmp_str2(rsc21_tab2["plx_str"][i])

rsc21_tab = Table([
    rsc21_tab2["psr_name"], rsc21_tab1["col2"], eph, ra, dec, pmra, pmdec, plx, ra_err,
    dec_err, pmra_err, pmdec_err, plx_err, ref, comment
],
    names=[
    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
    "pmdec_err", "plx_err", "ref", "comment"
])

## Paper 35: [Study of 72 pulsars discovered in the PALFA survey: Timing analysis, glitch activity, emission variability, and a pulsar in an eccentric binary](https://ui.adsabs.harvard.edu/abs/2021arXiv210802320P/abstract) 

by Parent, E. ; Sewalls, H. ; Freire, P. C. C. ; Matheny, T. ; Lyne, A. G. ; Perera, B. B. P. ; Cardoso, F. ; McLaughlin, M. A. ; Allen, B. ; Brazier, A. ; Camilo, F. ; Chatterjee, S. ; Cordes, J. M. ; Crawford, F. ; Deneva, J. S. ; Dong, F. A. ; Ferdman, R. D. ; Fonseca, E. ; Hessels, J. W. T. ; Kaspi, V. M. ; ...

The timing solution is requested directly from the first author.

The timing analysis uses **DE436**.

In [39]:
psf21_tab = Table.read("../data/palfa.dat", format="ascii")

psr_num = len(psf21_tab)

ref = ["psf+21"] * psr_num
comment = ["PALFA"] * psr_num

ref = Column(ref, name="ref")
comment = Column(comment, name="comment")

psf21_tab.add_columns([ref, comment])

Here I list below the papers which containting timing solution in a meachine readable format.

The references are listed below.

| Ref.Code    | Title   | Bibcode   |
| ----------- | :-----------: | ----------- |
| smd93     | [Timing parameters for 59 pulsars.](https://ui.adsabs.harvard.edu/abs/1993MNRAS.262..449S/abstract) | 1993MNRAS.262..449S | 
| bhl+94    | [Discovery of Three Binary Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/1994ApJ...425L..41B/abstract) | 1994ApJ...425L..41B | 
| ant94    | [Timing Behavior of 96 Radio Pulsars](https://ui.adsabs.harvard.edu/abs/1994ApJ...422..671A/abstract) | 1994ApJ...422..671A | 
| jml+95    | [Timing measurements for 45 pulsars.](https://ui.adsabs.harvard.edu/abs/1995A&A...293..795J/abstract) | 1995A&A...293..795J | 
| dgm+96    | [The New Northern Cross Pulsar System: Four Years of Pulsar Timing Observations](https://ui.adsabs.harvard.edu/abs/1996ApJS..106..611D/abstract) | 1996ApJS..106..611D | 
| dsb+98    | [The Parkes Southern Pulsar Survey - III. Timing of long-period pulsars](https://ui.adsabs.harvard.edu/abs/1998MNRAS.297...28D/abstract) | 1998MNRAS.297...28D | 
| idb98     | [New timing parameters and positions for 16 southern radio pulsars.](https://ui.adsabs.harvard.edu/abs/1998MNRAS.295..397I/abstract) | 1998MNRAS.295..397I | 
| tsb+99    | [Millisecond pulsar velocities](https://ui.adsabs.harvard.edu/abs/1999MNRAS.307..925T/abstract) | 1999MNRAS.307..925T | 
| wmz+01    | [Pulsar timing at Urumqi Astronomical Observatory: observing system and results](https://ui.adsabs.harvard.edu/abs/2001MNRAS.328..855W/abstract) | 2001MNRAS.328..855W | 
| mlc+01    | [The Parkes multi-beam pulsar survey - I. Observing and data analysis systems, discovery and timing of 100 pulsars](https://ui.adsabs.harvard.edu/abs/2001MNRAS.328...17M/abstract) | 2001MNRAS.328...17M | 
| kmb+03    | [The Parkes Multibeam Pulsar Survey - III. Young pulsars and the discovery and timing of 200 pulsars](https://ui.adsabs.harvard.edu/abs/2003MNRAS.342.1299K/abstract) | 2003MNRAS.342.1299K | 
| hlk+04    | [Long-term timing observations of 374 pulsars.](https://ui.adsabs.harvard.edu/abs/2004MNRAS.353.1311H/abstract) | 2004MNRAS.353.1311H | 
| hfs+04    | [The Parkes multibeam pulsar survey - IV. Discovery of 180 pulsars and parameters for 281 previously known pulsars](https://ui.adsabs.harvard.edu/abs/2004MNRAS.352.1439H/abstract) |  2004MNRAS.352.1439H | 
| zhw+05    | [Timing measurements and proper motions of 74 pulsars using the Nanshan radio telescope.](https://ui.adsabs.harvard.edu/abs/2005MNRAS.362.1189Z/abstract) | 2005MNRAS.362.1189Z | 
| lfl+06    | [The Parkes Multibeam Pulsar Survey - VI. Discovery and timing of 142 pulsars and a Galactic population analysis.](https://ui.adsabs.harvard.edu/abs/2006MNRAS.372..777L/abstract) | 2006MNRAS.372..777L | 
| hbo06    | [High-precision baseband timing of 15 millisecond pulsars](https://ui.adsabs.harvard.edu/abs/2006MNRAS.369.1502H/abstract) | 2006MNRAS.369.1502H | 
| mfl+06    | [Discovery of 14 Radio Pulsars in a Survey of the Magellanic Clouds](https://ui.adsabs.harvard.edu/abs/2006ApJ...649..235M/abstract)  | 2006ApJ...649..235M | 
| vbc+09    | [Timing stability of millisecond pulsars and prospects for gravitational-wave detection](https://ui.adsabs.harvard.edu/abs/2009MNRAS.400..951V/abstract) | 2009MNRAS.400..951V | 
| rkp+11    | [Precise γ-ray Timing and Radio Observations of 17 Fermi γ-ray Pulsars](https://ui.adsabs.harvard.edu/abs/2011ApJS..194...17R/abstract) | 2011ApJS..194...17R | 
| dfg+13    | [Limits on the Stochastic Gravitational Wave Background from the North American Nanohertz Observatory for Gravitational Waves](https://ui.adsabs.harvard.edu/abs/2013ApJ...762...94D/abstract) | 2013ApJ...762...94D | 
| slr+14    | [The Green Bank Northern Celestial Cap Pulsar Survey. I. Survey Description, Data Analysis, and Initial Results](https://ui.adsabs.harvard.edu/abs/2014ApJ...791...67S/abstract) | 2014ApJ...791...67S | 
| lwy+16    | [Proper motions of 15 pulsars: a comparison between Bayesian and frequentist algorithms.](https://ui.adsabs.harvard.edu/abs/2016MNRAS.460.4011L/abstract) | 2016MNRAS.460.4011L | 
| dcl+16    | [High-precision timing of 42 millisecond pulsars with the European Pulsar Timing Array](https://ui.adsabs.harvard.edu/abs/2016MNRAS.458.3341D/abstract) | 2016MNRAS.458.3341D | 
| mnf+16    | [The NANOGrav Nine-year Data Set: Astrometric Measurements of 37 Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/2016ApJ...818...92M/abstract) | 2016ApJ...818...92M | 
| vlh+16    | [The International Pulsar Timing Array: First data release](https://ui.adsabs.harvard.edu/abs/2016MNRAS.458.1267V/abstract) | 2016MNRAS.458.1267V | 
| rhc+16    | [Timing analysis for 20 millisecond pulsars in the Parkes Pulsar Timing Array](https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.1751R/abstract) | 2016MNRAS.455.1751R | 
| abb+18    | [The NANOGrav 11-year Data Set: High-precision Timing of 45 Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/2018ApJS..235...37A/abstract) | 2018ApJS..235...37A | 
| jbs+19    | [The UTMOST pulsar timing programme I: Overview and first results](https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.3691J/abstract) | 2019MNRAS.484.3691J | 
| psj+19    | [Timing of young radio pulsars - I. Timing noise, periodic modulation, and proper motion](https://ui.adsabs.harvard.edu/abs/2019MNRAS.489.3810P/abstract) | 2019MNRAS.489.3810P | 
| pdd+19    | [The International Pulsar Timing Array: second data release](https://ui.adsabs.harvard.edu/abs/2019MNRAS.490.4666P/abstract) | 2019MNRAS.490.4666P | 
| lbs+20    | [The UTMOST pulsar timing programme - II. Timing noise across the pulsar population](https://ui.adsabs.harvard.edu/abs/2020MNRAS.494..228L/abstract) | 2020MNRAS.494..228L | 
| dym+20    | [Results of 12 yr of Pulsar Timing at Nanshan. I.](https://ui.adsabs.harvard.edu/abs/2020ApJ...896..140D/abstract) | 2020ApJ...896..140D | 
| lys+20    | [New timing measurement results of 16 pulsars](https://ui.adsabs.harvard.edu/abs/2020PASJ...72...70L/abstract) | 2020PASJ...72...70L | 
| rsc+21    | [The Parkes pulsar timing array second data release: timing analysis](https://ui.adsabs.harvard.edu/abs/2021MNRAS.507.2137R/abstract) | 2021MNRAS.507.2137R | 
| psf+21    | [Study of 72 pulsars discovered in the PALFA survey: Timing analysis, glitch activity, emission variability, and a pulsar in an eccentric binary](https://ui.adsabs.harvard.edu/abs/2021arXiv210802320P/abstract) | 2021arXiv210802320P | 



There are also measurement for individual pulsars.

The references are listed below.

| Ref.Code    | Title   | Bibcode   | Comment   |
| ----------- | :-----------: | ----------- |----------- |
| aft94     | [Orbital Variability in the Eclipsing Pulsar Binary PSR B1957+20](https://ui.adsabs.harvard.edu/abs/1994ApJ...426L..85A/abstract) | 1994ApJ...426L..85A | 1 PSR |
| cst96     | [Princeton-Arecibo Declination-Strip Survey for Millisecond Pulsars. I.](https://ui.adsabs.harvard.edu/abs/1996ApJ...469..819C/abstract) | 1996ApJ...469..819C | 2 PSRs |
| kmb+96    | [Evidence from a precessing pulsar orbit for a neutron-star birth kick](https://ui.adsabs.harvard.edu/abs/1996Natur.381..584K/abstract) | 1996Natur.381..584K | 1 PSRs |
| bbm+97     | [Timing Measurements and Their Implications for Four Binary Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/1997MNRAS.286..463B/abstract) | 1997MNRAS.286..463B | 4 PSRs |
| sac+98     | [Measurement of Relativistic Orbital Decay in the PSR B1534+12 Binary System](https://ui.adsabs.harvard.edu/abs/1998ApJ...505..352S/abstract) | 1998ApJ...505..352S | 1 PSR |
| kxc+99    | [Profile Instabilities of the Millisecond Pulsar PSR J1022+1001](https://ui.adsabs.harvard.edu/abs/1999ApJ...520..324K/abstract) | 1999ApJ...520..324K | ecliptic coordinate (Not used) |
| wdk+00    | [Timing Observations of Four Millisecond Pulsars with the Arecibo and Effelsberg Radio Telescopes](https://ui.adsabs.harvard.edu/abs/2000ApJ...528..907W/abstract) | 2000ApJ...528..907W | 4 PSRs |
| clm+01    | [Discovery of Five Binary Radio Pulsars](https://ui.adsabs.harvard.edu/abs/2001ApJ...548L.187C/abstract) | 2001ApJ...548L.187C | 5 PSRs |
| lcw+01    | [Precision timing measurements of PSR J1012+5307](https://ui.adsabs.harvard.edu/abs/2001MNRAS.326..274L/abstract) | 2001MNRAS.326..274L | 1 PSR |
| eb01      | [Recycled Pulsars Discovered at High Radio Frequency](https://ui.adsabs.harvard.edu/abs/2001ApJ...553..801E/abstract) |  2001ApJ...553..801E | 8 PSRs |
| stt+02      | [Studies of the Relativistic Binary Pulsar PSR B1534+12. I. Timing Analysis](https://ui.adsabs.harvard.edu/abs/2002ApJ...581..501S/abstract) |  2002ApJ...581..501S | 1 PSR |
| kwh03      | [Geodetic Precession and Timing of the Relativistic Binary Pulsars PSR B1534+12 and PSR B1913+16](https://ui.adsabs.harvard.edu/abs/2003ApJ...589..495K/abstract) |  2003ApJ...589..495K | 1 PSR |
| lkd+04    | [The parallax, mass and age of the PSR J2145-0750 binary system](https://ui.adsabs.harvard.edu/abs/2004A&A...426..631L/abstract) | 2004A&A...426..631L | PM in ecliptic coordinate (transformed) |
| jml+09    | [Discovery of three new pulsars in a 610-MHz pulsar survey with the GMRT.](https://ui.adsabs.harvard.edu/abs/2009MNRAS.398..943J/abstract) | 2009MNRAS.398..943J | No common PSR |
| jsb+09    | [Discovery and timing of the first 8gr8 Cygnus survey pulsars](https://ui.adsabs.harvard.edu/abs/2009A&A...498..223J/abstract) | 2009A&A...498..223J | 3 PSRs |
| lwj+09    | [Generic tests of the existence of the gravitational dipole radiation and the variation of the gravitational constant](https://ui.adsabs.harvard.edu/abs/2009MNRAS.400..805L/abstract) | 2009MNRAS.400..805L | 1 PSRs |
| gsf+11    | [High-precision Timing of Five Millisecond Pulsars: Space Velocities, Binary Evolution, and Equivalence Principles](https://ui.adsabs.harvard.edu/abs/2011ApJ...743..102G/abstract) | 2011ApJ...743..102G | 5 PSRs |
| pgf+12    | [Binary Millisecond Pulsar Discovery via Gamma-Ray Pulsations](https://ui.adsabs.harvard.edu/abs/2012Sci...338.1314P/abstract) | 2012Sci...338.1314P | 1 PSR |
| afw+13    | [A Massive Pulsar in a Compact Relativistic Binary](https://ui.adsabs.harvard.edu/abs/2013Sci...340..448A/abstract) | 2013Sci...340..448A | 1 PSR |
| cls+13    | [PSR J1723-2837: An Eclipsing Binary Radio Millisecond Pulsar](https://ui.adsabs.harvard.edu/abs/2013ApJ...776...20C/abstract) | 2013ApJ...776...20C | 1 PSR |
| fst14    | [A Comprehensive Study of Relativistic Gravity Using PSR B1534+12](https://ui.adsabs.harvard.edu/abs/2014ApJ...787...82F/abstract) | 2014ApJ...787...82F | 1 PSR |
| sjm14    | [The kinematics and orbital dynamics of the PSR B1259-63/LS 2883 system from 23 yr of pulsar timing](https://ui.adsabs.harvard.edu/abs/2014MNRAS.437.3255S/abstract) | 2014MNRAS.437.3255S | 1 PSR |
| btb+15    | [The High Time Resolution Universe survey - XI. Discovery of five recycled pulsars and the optical detectability of survey white dwarf companions](https://ui.adsabs.harvard.edu/abs/2015MNRAS.446.4019B/abstract) | 2015MNRAS.446.4019B | Unknown DE version, assumed DE421 | 
| rbb+15   | [Discovery of Psr J1227-4853: A Transition from a Low-mass X-Ray Binary to a Redback Millisecond Pulsar](https://ui.adsabs.harvard.edu/abs/2015ApJ...800L..12R/abstract) | 2015ApJ...800L..12R | 1 PSR |
| sab+16   | [Timing of Five PALFA-discovered Millisecond Pulsars](https://ui.adsabs.harvard.edu/abs/2016ApJ...833..192S/abstract) | 2016ApJ...833..192S | 5 PSRs |
| hnl+17   | [Multiwavelength monitoring and X-ray brightening of Be X-ray binary PSR J2032+4127/MT91 213 on its approach to periastron](https://ui.adsabs.harvard.edu/abs/2017MNRAS.464.1211H/abstract) | 2017MNRAS.464.1211H | 1 PSR |
| pkr+19    | [Eight Millisecond Pulsars Discovered in the Arecibo PALFA Survey](https://ui.adsabs.harvard.edu/abs/2019ApJ...886..148P/abstract) | 2019ApJ...886..148P | 8 PSRs | 
| mpb+20    | [NuSTAR and Parkes observations of the transitional millisecond pulsar binary XSS J12270-4859 in the rotation-powered state](https://ui.adsabs.harvard.edu/abs/2020MNRAS.492.5607D/abstract) | 2020MNRAS.492.5607D| 1 PSR | 
| drc+21    | [Timing of Eight Binary Millisecond Pulsars Found with Arecibo in Fermi-LAT Unidentified Sources](https://ui.adsabs.harvard.edu/abs/2021ApJ...909....6D/abstract) | 2021ApJ...909....6D | 8 PSRs | 

In [40]:
dat_tab = [
    # aft94
    [
        "B1957+20", 48196.0, "DE200", "19:59:36.76988(5)", "+20:48:15.1222(6)",
        "-16.0(5)", "-25.8(6)", "", "aft94", ""
    ],
    # cst96
    [
        "J0621+1002", 49950.0, "DE200", "06:21:22.1103(6)", "+10:02:38.79(4)",
        "", "", "", "cst96", ""
    ],
    [
        "J1022+1001", 49780.0, "DE200", "10:22:58.06(6)", "+10:01:54(3)", "",
        "", "", "cst96", ""
    ],
    # kmb+96
    [
        "J0045-7319", 49144.0000, "DE200", "00 45 35.16(7)", "-73 19 03.0(2)",
        "", "", "", "kmb+96", ""
    ],
    # bbm+97
    [
        "J0437-4715", 49615.0, "DE200", "04:37:15.73501(2)",
        "-47:15:08.1527(2)", "121.3(2)", "-70.4(3)", "1(5)", "bbm+97", ""
    ],
    [
        "J0613-0200", 49512.0, "DE200", "06:13:43.97317(14)",
        "-02:00:47.072(6)", "1.5(30)", "-4(6)", "", "bbm+97", ""
    ],
    [
        "J1045-4509", 49597.0, "DE200", "10:45:50.91940(3)",
        "-45:09:54.204(2)", "-7(3)", "8(2)", "", "bbm+97", ""
    ],
    [
        "J1643-1224", 49524.0, "DE200", "16:43:38.1548(2)",
        "-12:24:58.740(14)", "7(3)", "7(17)", "", "bbm+97", ""
    ],
    # sac+98
    [
        "B1534+12", 48778.0, "DE200", "15:37:09.95994(2)", "11:55:55.6561(3)",
        "1.3(3)", "-25.5(4)", "", "sac+98", ""
    ],
    # wdk+00
    [
        "B1257+12", 48700.0, "DE200", "13:00:03.04865(1)", "12:40:56.7134(3)",
        "46.4(1)", "82.2(2)", "1.3(4)", "wdk+00", ""
    ],
    [
        "J1640+2224", 49360.0, "DE200", "16:40:16.74173(3)",
        "22:24:09.0113 (6)", "-0.1 (3)", "-13.3 (7)", "", "wdk+00", ""
    ],
    [
        "B1953+29", 49718.0, "DE200", "19:55:27.8757 (2) ", "29:08:43.530 (2)",
        "-1.0 (3)", "-3.7 (3) ", "", "wdk+00", ""
    ],
    [
        "J2229+2643", 49718.0, "DE200", "22:29:50.8864 (2)",
        "26:43:57.785 (4)", "1 (4)", "-17 (4)", "", "wdk+00", ""
    ],
    # clm+01
    [
        "J1232-6501", 51270.0, "DE200", "12 32 17.840(5)", "-65 01 03.33(4)",
        "", "", "", "clm+01", ""
    ],
    [
        "J1435-6100", 51270.0, "DE200", "14 35 20.2765(4)", "-61 00 57.956(6)",
        "", "", "", "clm+01", ""
    ],
    [
        "J1454-5846", 51300.0, "DE200", "14 54 10.908(2)", "-58 46 34.74(3)",
        "", "", "", "clm+01", ""
    ],
    [
        "J1810-2005", 51200.0, "DE200", "18 10 58.988(2)", "-20 05 08.3(6)",
        "", "", "", "clm+01", ""
    ],
    [
        "J1904+0412", 51450.0, "DE200", "19 04 31.382(4)", "+04 12 05.9(1)",
        "", "", "", "clm+01", ""
    ],
    # lcw+01
    [
        "J1012+5307", 50700.0, "DE200", "10:12:33.43370(3)",
        "53:07:02.5884(4)", "2.4(2)", "-25.2(2)", "", "lcw+01", ""
    ],
    # eb01
    ["J1157-5112", 51400.0, "DE200", "11:57:08.166(1)", "-51:12:56.14(3)", "", "", "", "eb01", ""],
    ["J1410-7404", 51460.0, "DE200", "14:10:07.370(5)", "-74:04:53.32(2)", "", "", "", "eb01", ""],
    ["J1629-6902", 51600.0, "DE200", "16:29:08.7706(4)", "-69:02:45.294(3)", "", "", "", "eb01", ""],
    ["J1721-2457", 51600.0, "DE200", "17:21:05.496(2)", "-24:57:06.1(4)", "", "", "", "eb01", ""],
    ["J1745-0952", 51500.0, "DE200", "17:45:09.1400(9)", "-09:52:39.67(5)", "", "", "", "eb01", ""],
    ["J1732-5049", 51575.0, "DE200", "17:32:47.7671(4)", "-50:49:00.11(1)", "", "", "", "eb01", ""],
    ["J1757-5322", 51570.0, "DE200", "17:57:15.1615(3)", "-53:22:26.387(5)", "", "", "", "eb01", ""],
    ["J1918-0642", 51600.0, "DE200", "19:18:48.0363(8)", "-06:42:34.80(5)", "", "", "", "eb01", ""],
    # stt+02
    [
        "B1534+12", 50261.0, "DE200", "15:37:09.960312(10)", "11:55:55.5543(2)",
        "1.32(3)", "-25.12(5)", "", "stt+02", ""
    ],
    # kwh03
    [
        "B1534+12", 50300.0, "DE200", "15:37:09.960323(3)", "11:55:55.55186(7)",
        "1.34(1)", "-25.05(2)", "", "kwh03", ""
    ],
    # lkd+04
    [
        "J2145−0750", 50800.0, "DE200", "21:45:50.46726(10)",
        "−07:50:18.375(8)", "-14.05(28)", "-1.3(6)", "", "lkd+04", ""
    ],
    # jml+09
    [
        "J0026+6320", 54000.0, "DE200", "00:26:50.561(8)", "63:20:00.87(5)",
        "", "", "", "jml+09", ""
    ],
    [
        "J2208+5500", 54315.0, "DE200", "22:08:23.72(1)", "55:00:08.41(5)", "",
        "", "", "jml+09", ""
    ],
    [
        "J2217+5733", 54315.0, "DE200", "22:17:55.03(2)", "57:33:04.0(2)", "",
        "", "", "jml+09", ""
    ],
    # jsb+09
    [
        "J1937+2950", 54250.0, "DE405", "19h37m47.603(14)", "+29d50m01.8(2)",
        "", "", "", "jsb+09", ""
    ],
    [
        "J2027+4557", 54250.0, "DE405", "20h27m16.233(3)", "+45d57m57.08(4)",
        "", "", "", "jsb+09", ""
    ],
    [
        "J2047+5029", 54250.0, "DE405", "20h47m54.6400(4)", "+50d29m38.17(4)",
        "", "", "", "jsb+09", ""
    ],
    # lwj+09
    [
        "J1012+5307", 50700.0, "DE405", "10h12m33.4341010(99)",
        "53d07m02.60070(13)", "2.562(14)", "-25.61(2)", "1.22(26)", "lwj+09",
        ""
    ],
    # gsf+11
    [
        "J1853+1303", 54000, "DE405", "18:53:57.319174(8)", "13:03:44.0784(2)",
        "-1.68(7)", "-2.94(12)", "1.0(6)", "gsf+11", ""
    ],
    [
        "J1905+0400", 53700, "DE405", "19:05:28.273436(16)",
        "04:00:10.8830(6)", " -3.80(18)", "-7.3(4)", "", "gsf+11", ""
    ],
    [
        "J1910+1256", 54000, "DE405", "19:10:09.701479(8)", "12:56:25.5074(3)",
        "0.21(10)", "-7.25(12)", "", "gsf+11", ""
    ],
    [
        "B1953+29", 54500, "DE405", "19:55:27.87600(3)", "29:08:43.4659(5)",
        "-0.9(1)", "-4.1(1)", "", "gsf+11", ""
    ],
    [
        "J2016+1948", 53000.0, "DE405", "20:16:57.44349(6)",
        "19:47:51.5882(12)", "1.28(26)", "2.83(34)", "", "gsf+11", ""
    ],
    # pgf+12
    [
        "J1311-3430", 55266.90789575858, "DE405", "13:11:45.7242(2)",
        "-34:30:30.350(4)", "", "", "", "pgf+12", ""
    ],
    # afw+13
    [
        "J0348+0432", 56000, "DE421", "03:48:43.639000(4)",
        "+04:32:11.4580(2)", "+4.04(16)", "+3.5(6)", "", "afw+13", ""
    ],
    # cls+13
    [
        "J1723-2837", 55667, "DE405", "17:23:23.1856(8)", "-28:37:57.17(11)",
        "", "", "", "cls+13", ""
    ],
    # fst14 (B1534+12)
    [
        "B1534+12", 52077, "DE421", "15:37:09.961730(3)", "11:55:55.43387(6)",
        "1.482(7)", "-25.285(12)", " 0.86(18)", "fst14", ""
    ],
    # sjm14 (B1259-63)
    [
        "J1302-6350", 55000, "DE421", "13:02:47.6426(11)", "-63:50:08.665(8)",
        "-6.6(1.8)", "-4.4(1.4)", "", "sjm14", ""
    ],
    # btb+15
    # Since papers X and XVI in the series used DE421, 
    # I assume that paper XI also used DE421
    [
        "J1227-6208", 55991.2, "DE421", "12:27:00.4413(4)", "-62:08:43.789(3)",
        "", "", "", "btb+15", ""
    ],
    [
        "J1405-4656", 55692.7, "DE421", "14:05:21.4255(8)", "-46:56:02.31(1)",
        "-44(6)", "20(10)", "", "btb+15", ""
    ],
    [
        "J1431-4715", 55756.1, "DE421", "14:31:44.6177(2)", "-47:15:27.574(4)",
        "-7(3)", "-8(4)", "", "btb+15", ""
    ],
    [
        "J1653-2054", 55694.6, "DE421", "16:53:31.03(2)", "-20:54:55.1(1) ",
        "", "", "", "btb+15", ""
    ],
    [
        "J1729-2117", 55505, "DE421", "17:29:10.808(6)", "-21:17:28(1)", "",
        "", "", "btb+15", ""
    ],
    # rbb+15
    [
        "J1227-4853", 56707.9764, "DE405", "12:27:58.724(1)",
        "-48:53:42.741(3)", "", "", "", "rbb+15", ""
    ],
    # sab+16
    [
        "J1906+0055", 56408.0, "DE421", "19:06:48.68051(4)", "00:55:07.886(1)",
        "", "", "", "sab+16", "PALFA"
    ],
    [
        "J1914+0659", 56351.0, "DE421", "19:14:17.647(2)", "07:01:11.00(7)",
        "", "", "", "sab+16", "PALFA"
    ],
    [
        "J1933+1726", 56466.0, "DE421", "19:33:22.9828(3)", "17:26:49.606(9)",
        "", "", "", "sab+16", "PALFA"
    ],
    [
        "J1938+2012", 56511.0, "DE421", "19:38:40.0803(1)", "20:12:50.827(3)",
        "", "", "", "sab+16", "PALFA"
    ],
    [
        "J1957+2516", 56408.0, "DE421", "19:57:34.6115(3)", "25:16:02.076(3)",
        "", "", "", "sab+16", "PALFA"
    ],
    # hnl+17
    [
        "J2032+4127", 55700.0, "DE421", "20h32m13.119(2)", "41d27m24.34(2)",
        "", "", "", "hnl+17", ""
    ],
    # pkr+19
    [
        "J1906+0454", 58100.0, "DE421", "19:06:47.13970(7)",
        "+04:54:09.042(2)", "-6(1)", "-14(3)", "", "pkr+19", "PALFA"
    ],
    [
        "J1913+0618", 57696.0, "DE421", "19:13:17.8786(2)", "+06:18:08.277(5)",
        "-9(2)", "", "", "pkr+19", "PALFA"
    ],
    [
        "J1921+1929", 58137.0, "DE421", "19:21:23.35070(4)",
        "+19:29:22.299(1)", "-3.2(8)", "-11(1)", "", "pkr+19", "PALFA"
    ],
    [
        "J1928+1245", 57974.0, "DE421", "19:28:45.39360(6)",
        "+12:45:53.374(3)", "", "", "", "pkr+19", "PALFA"
    ],
    [
        "J1930+2441", 58177.0, "DE421", "19:30:45.39844(6)",
        "+24:41:01.564(1)", "", "-10(3)", "16(6)", "pkr+19", "PALFA"
    ],
    [
        "J1932+1756", 57516.0, "DE421", "19:32:07.2277(5)", "+17:56:17.31(1)",
        "-13(5)", "", "", "pkr+19", "PALFA"
    ],
    [
        "J1937+1658", 57516.0, "DE421", "19:37:24.90851(8)",
        "+16:58:42.553(2)", "-1.8(7)", "-5(1)", "", "pkr+19", "PALFA"
    ],
    [
        "J2010+3051", 57949.0, "DE421", "20:10:03.9542(1)", "+30:51:18.892(1)",
        "-10.3(9)", "-3(1)", "", "pkr+19", "PALFA"
    ],
    # mpb+20
    [
        "J1227–4853", 57180, "DE405", "12:27:58.7186(4)", "−48:53:42.707(5)",
        "", "", "", "mpb+20", ""
    ],
    [
        "J1227–4853", 57139, "DE405", "12:27:58.7194(14)", "−48:53:42.712(19)",
        "", "", "", "mpb+20", ""
    ],
    # drc+21
    [
        "J1625-0021", 57114, "DE421", "16:25:10.3579(3)", "-00:21:28.960(8)",
        "-3.6(13)", "-8.9(42)", "", "drc+21", ""
    ],
    [
        "J1824+1014", 56925, "DE421", "18:24:14.93368(3)", "+10:14:43.82(3)",
        "", "", "", "drc+21", ""
    ],
    [
        "J2006+0148", 57243, "DE421", "20:06:29.0529(2)", "+01:48:53.919(7)",
        "", "", "", "drc+21", ""
    ],
    [
        "J0251+2606", 56644, "DE421", "02:51:02.5540(7)", "+26:06:09.97(2)",
        "17(3)", "", "", "drc+21", ""
    ],
    [
        "J1805+0615", 57002, "DE421", "18:05:42.39969(100)",
        "+06:15:18.6148(20)", "8.7(13)", "12.8(29)", "", "drc+21", ""
    ],
    [
        "J1908+2105", 56930, "DE421", "19:08:57.2939(4)", "+21:05:02.720(8)",
        "", "", "", "drc+21", ""
    ],
    [
        "J2052+1219", 56548, "DE421", "20:52:47.77818(30)",
        "+12:19:59.0204(6)", "-4.30(32)", "", "", "drc+21", ""
    ],
    [
        "J1048+2339", 56700, "DE421", "10:48:43.418354(76)",
        "+23:39:53.4043(20)", "", "", "", "drc+21", ""
    ],
]

psr_num = len(dat_tab)
psr_name = []
pos_epoch = np.zeros(psr_num)
eph = []
ra = np.zeros(psr_num)
dec = np.zeros(psr_num)
pmra = np.zeros(psr_num)
pmdec = np.zeros(psr_num)
ra_err = np.zeros(psr_num)
dec_err = np.zeros(psr_num)
pmra_err = np.zeros(psr_num)
pmdec_err = np.zeros(psr_num)
plx = np.zeros(psr_num)
plx_err = np.zeros(psr_num)
ref = []
comment = []

for i in range(psr_num):
    
    psr_namei = dat_tab[i][0]
#     print(dat_tab[i])
    if psr_namei in psr_name_map.keys():
        psr_name.append(psr_name_map[psr_namei])
    else:
        psr_name.append(psr_namei)

    pos_epoch[i] = dat_tab[i][1]
    eph.append(dat_tab[i][2])

    ra[i], dec[i], ra_err[i], dec_err[i] = parse_radec_str(
        dat_tab[i][3], dat_tab[i][4])

    pmra[i], pmra_err[i] = parse_pmp_str2(dat_tab[i][5])
    pmdec[i], pmdec_err[i] = parse_pmp_str2(dat_tab[i][6])
    plx[i], plx_err[i] = parse_pmp_str2(dat_tab[i][7])

    ref.append(dat_tab[i][8])
    comment.append(dat_tab[i][9])

oth_tab = Table([
    psr_name, pos_epoch, eph, ra, dec, pmra, pmdec, plx, ra_err, dec_err,
    pmra_err, pmdec_err, plx_err, ref, comment
],
                names=[
                    "psr_name", "pos_epoch", "eph", "ra", "dec", "pmra",
                    "pmdec", "plx", "ra_err", "dec_err", "pmra_err",
                    "pmdec_err", "plx_err", "ref", "comment"
                ])

# 2 Find data for EDR3 and PSRPI

Concatenate all data and do the cross-match.

In [41]:
timing_psr = vstack((
    smd93_tab,
    bhl94_tab,
    jml95_tab,
    dgm96_tab,
    idb98_tab,
    dsb98_tab,
    tsb99_tab,
    wmz01_tab,
    mlc01_tab,
    kbm03_tab,
    hlk04_tab,
    hfs04_tab,
    zhw05_tab,
    lfl06_tab,
    hbo06_tab,
    mfl06_tab,
    vbc09_tab,
    rkp11_tab,
    dfg13_tab,
    slr14_tab,
    lwy16_tab,
    dcl16_tab,
    mnf16_tab,
    vlh16_tab,
    rhc16_tab,
    abb18_tab,
    jbs19_tab,
    psj19_tab,
    pdd19_tab,
    lbs20_tab,
    dym20_tab,
    rsc21_tab,
    psf21_tab,
    oth_tab,
))

timing_psrb = vstack((
    ant94_tab,
    lys20_tab,
))

In [42]:
timing_edr3 = join(edr3_psr, timing_psr, keys="psr_name")  # Nb. Common = 70
timingb_edr3 = join(edr3_psr, timing_psrb, keys="psr_bname")
timing_edr3 = vstack((timing_edr3, timingb_edr3))

timing_edr3.write("../data/timing_edr3.fits", overwrite=True)

In [43]:
timing_vlbi = join(vlbi_psr, timing_psr, keys="psr_name") # Nb. Common = 209
timingb_vlbi = join(vlbi_psr, timing_psrb, keys="psr_bname")
timing_vlbi = vstack((timing_vlbi, timingb_vlbi))

timing_vlbi.write("../data/timing_vlbi.fits", overwrite=True)